In [49]:
import numpy as np
from PIL import Image
from collections import Counter

In [50]:
img_name = input("Enter Image Name with extension: ")
original_image_array = np.array(Image.open(img_name))
height, width, channels = original_image_array.shape

In [51]:
# Creating separate images for R, G, B channels
Red_Image = np.stack([original_image_array[:,:,0]]*3, axis=-1)
Green_Image = np.stack([original_image_array[:,:,1]]*3, axis=-1)
Blue_Image = np.stack([original_image_array[:,:,2]]*3, axis=-1)

# Convert arrays to images, and save or display
red_img = Image.fromarray(Red_Image)
green_img = Image.fromarray(Green_Image)
blue_img = Image.fromarray(Blue_Image)

# Display the images
red_img.show()
green_img.show()
blue_img.show()

# Save the images
red_img.save('Red_Image.png')
green_img.save('Green_Image.png')
blue_img.save('Blue_Image.png')

In [52]:
red_with_noise = np.array(Image.open("Red_Image_With_Noise.png"))
green_with_noise = np.array(Image.open("Green_Image_With_Noise.png"))
blue_with_noise = np.array(Image.open("Blue_Image_With_Noise.png"))

In [53]:
def convert_to_binary(image_array):
    # Get dimensions
    height, width, _ = image_array.shape
    
    # Initialize binary image array
    binary_image = np.zeros((height, width, 3), dtype='U8')

    # Convert each RGB value to a binary string
    for i in range(height):
        for j in range(width):
            binary_image[i, j, 0] = format(image_array[i, j, 0], '08b')  # Channel 1
            binary_image[i, j, 1] = format(image_array[i, j, 1], '08b')  # Channel 2
            binary_image[i, j, 2] = format(image_array[i, j, 2], '08b')  # Channel 3

    return binary_image

# Convert noisy images to binary
binary_red_with_noise = convert_to_binary(red_with_noise)
binary_green_with_noise = convert_to_binary(green_with_noise)
binary_blue_with_noise = convert_to_binary(blue_with_noise)

In [54]:
def majority_vote(bit_lists):
    voted_binary_string = ''
    
    for i in range(8):
        # Collect the specific bit from each binary string in the list at position i
        bit_column = [bits[i] for bits in bit_lists]
        
        # Use a Counter to count occurrences of each bit in the current position
        bit_counter = Counter(bit_column)
        
        # Find the most common bit at the current position
        most_common_bit, _ = bit_counter.most_common(1)[0]
        
        voted_binary_string += most_common_bit

    return voted_binary_string

In [59]:
# Initialize the final image array
final_image_array = np.zeros((height, width, 3), dtype=np.uint8)

height, width, _ = red_with_noise.shape

# Iterate over each pixel
for i in range(height):
    for j in range(width):
        # Extract the list of binary strings for the current pixel from each color channel
        red_bit_lists = binary_red_with_noise[i, j]
        green_bit_lists = binary_green_with_noise[i, j]
        blue_bit_lists = binary_blue_with_noise[i, j]
        
        # Perform majority voting for each color channel
        red_voted_binary_string = majority_vote(red_bit_lists)
        green_voted_binary_string = majority_vote(green_bit_lists)
        blue_voted_binary_string = majority_vote(blue_bit_lists)
        
        # Convert the voted binary strings back to integers and store them in the final image array
        final_image_array[i, j, 0] = int(red_voted_binary_string, 2)
        final_image_array[i, j, 1] = int(green_voted_binary_string, 2)
        final_image_array[i, j, 2] = int(blue_voted_binary_string, 2)


In [60]:
# Convert the numpy array to an image and save or display it
final_image = Image.fromarray(final_image_array, 'RGB')
final_image.save('Corrected_Image.png')
final_image.show()